In [1]:
library(tidyverse)
library(plotly)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.1     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:stats’:

    filter


The following object is masked from ‘package:graphics’:

    layout




## Download the data

In [2]:
quakes = read_csv('https://raw.githubusercontent.com/plotly/datasets/master/earthquakes-23k.csv')

quakes = quakes |> 
  mutate(Date = parse_date_time(Date, orders = "%m/%d/%Y"))

head(quakes)

Rows: 23412 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): Date
dbl (3): Latitude, Longitude, Magnitude

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Date = parse_date_time(Date, orders = "%m/%d/%Y")`.
Caused by warning:
!  3 failed to parse.”


Date,Latitude,Longitude,Magnitude
<dttm>,<dbl>,<dbl>,<dbl>
1965-01-02,19.246,145.616,6.0
1965-01-04,1.863,127.352,5.8
1965-01-05,-20.579,-173.972,6.2
1965-01-08,-59.076,-23.557,5.8
1965-01-09,11.938,126.427,5.8
1965-01-10,-13.405,166.629,6.7


## Inspect the DataFrame

In [3]:
skimr::skim(quakes)

── Data Summary ────────────────────────
                           Values
Name                       quakes
Number of rows             23412 
Number of columns          4     
_______________________          
Column type frequency:           
  numeric                  3     
  POSIXct                  1     
________________________         
Group variables            None  

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate  mean      sd     p0   p25    p50   p75
1 Latitude              0             1  1.68  30.1    -77.1 -18.7  -3.57  26.2
2 Longitude             0             1 39.6  126.    -180.  -76.3 104.   145. 
3 Magnitude             0             1  5.88   0.423    5.5   5.6   5.7    6  
   p100 hist 
1  86.0 ▂▆▇▅▁
2 180.  ▃▂▁▂▇
3   9.1 ▇▁▁▁▁

── Variable type: POSIXct ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate min                 max                

Warning message in is.null(text_repr) || nchar(text_repr) == 0L:
“'length(x) = 16 > 1' in coercion to 'logical(1)'”


,skim_type,skim_variable,n_missing,complete_rate,POSIXct.min,POSIXct.max,POSIXct.median,POSIXct.n_unique,numeric.mean,numeric.sd,numeric.p0,numeric.p25,numeric.p50,numeric.p75,numeric.p100,numeric.hist
,<chr>,<chr>,<int>,<dbl>,<dttm>,<dttm>,<dttm>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,POSIXct,Date,3,0.9998719,1965-01-02,2016-12-30,1993-11-30,12398,NA,NA,NA,NA,NA,NA,NA,NA
2,numeric,Latitude,0,1.0000000,NA,NA,NA,NA,1.679033,30.1131829,-77.080,-18.65300,-3.5685,26.19075,86.005,▂▆▇▅▁
3,numeric,Longitude,0,1.0000000,NA,NA,NA,NA,39.639961,125.5119585,-179.997,-76.34975,103.9820,145.02625,179.998,▃▂▁▂▇
4,numeric,Magnitude,0,1.0000000,NA,NA,NA,NA,5.882531,0.4230656,5.500,5.60000,5.7000,6.00000,9.100,▇▁▁▁▁


## Geospatial Analysis

In [ ]:
fig = quakes |>
  plot_ly(
    type = 'densitymapbox',
    lat = ~ Latitude,
    lon = ~ Longitude,
    coloraxis = 'coloraxis',
    radius = 5
  ) 

fig |>
  layout(
    mapbox = list(
      style = "stamen-terrain",
      center = list(lon = 180)), coloraxis = list(colorscale = "Viridis")
    )

## Examine temporal patterns

Plotting the number of earthquakes by year indicates that earthquakes are becoming more frequent:

In [ ]:
yearly = quakes |>
    group_by(Year = year(Date)) |>
    summarize(n = n())

yearly |>
  plotly::plot_ly(type = "bar",
                  x = ~ Year,
                  y = ~ n)

This is maybe unexpected. To look into the data more, we can also plot by time and magnitude. This shows that the number of large magnitude events stays relatively constant through time, and the apparent increase entirely comes from smaller magnitude events. This might suggest a detection bias.

In [ ]:
yearly_mag = quakes |> 
  group_by(Year = year(Date), Magnitude = cut_width(Magnitude, 1)) |> 
  summarize(n = n())

yearly_mag |> 
  plot_ly(
    type = "scatter",
    x = ~ Year,
    y = ~ n,
    color = ~ Magnitude,
    mode = "markers"
  )